# Autenticidad, 
# los Mensajes Secretos,
# y Seguridad



<a href="https://colab.research.google.com/github/ProfDoeg/Colegio_Invisible/blob/master/04_cuaderno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparar el Espacio (Prepare the Space)

In [ ]:
! pip install eciespy eth_keys gspread oauth2client
! git clone https://github.com/ProfDoeg/Colegio_Invisible.git
! pip install --upgrade gspread

In [ ]:
#rm llaves -r
#!rm llaves.zip
!ls

# Aplicar Criptografía Para Crear un Ensamblaje de Mensajería Seguro 
# (Apply Cryptography to Create a Secure Messaging Assembly)

## Esquema (Outline) 

Aplicar criptografía para crear un tablero de mensajes securos.
***
Apply cryptography to build a secure messaging board.

# Metas (Goals)

- crear y guardar claves\
(create and save keys)
- descargar/cargar claves\
(download/upload keys)
- descifrar el token de acceso a la hoja de cálculo\
(decrypt spreadsheet access token)
- entender `gspread`\
(understand `gspread`)
- publicar: nombre, clave pública, mensaje público y firma\
(post: name, public key, public message, and signature)
- firmar mensaje\
(sign message)
- verificar mensaje\
(verify message)
- escribir mensaje a otros participantes\
(write message to other participants)
- escribir un mensaje encriptado a otro participante\
(write encrypted message to other participant)
- escribir mensaje firmado y encriptado\
(write signed and encrypted message)


# Make New Keys

In [ ]:
! mkdir llaves


In [ ]:
! python Colegio_Invisible/scripts/ecc_generate.py llaves/prvkey1.eck

In [ ]:
! python Colegio_Invisible/scripts/ecc_pubkey_extract.py llaves/prvkey1.eck llaves/pubkey1.eck 

# Saving Keys

In [ ]:
#note
! zip -r llaves.zip llaves

In [ ]:
#note
from google.colab import files
f=files.download('llaves.zip');

# Uploading Keys

In [ ]:
from google.colab import files
fu=files.upload();

In [ ]:
! unzip llaves.zip

# Access Pubkey Using Script

In [ ]:
ret=!{ 'python Colegio_Invisible/scripts/ecc_keydump.py llaves/pubkey1.eck'}
pubkey1=ret[-1]
pubkey1

# `gspread` 


## Decrypt Credentials

colegioinvisible binary

Decrypt `client_secret.aes` using password

This will be communicated over voice by DrDoeg



In [ ]:
! python Colegio_Invisible/scripts/aes_decrypt.py Colegio_Invisible/secrets/client_secret.aes Colegio_Invisible/secrets/client_secret.json 

In [ ]:
! ls Colegio_Invisible/secrets

`gspread` is a python module for programmatic interaction with Google Sheets. 

All the participants will have access to the same spreadsheet in the cloud.

https://docs.gspread.org/en/latest/user-guide.html

We use the decrypted credentials to authenticate and gain API access to the spreadsheet

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('Colegio_Invisible/secrets/client_secret.json', scope)
client = gspread.authorize(creds)
sheets = client.open("signatures")

Here we can see the structure of the spreadsheet and all of the layers or `Worksheet`

In [ ]:
sheets.worksheets()

Isolating the first sheet where we query the first row

In [ ]:
sheet=sheets.worksheets()[0]
sheet.row_values(1)

In [ ]:
#sheet.clear()
#sheet.update_cell(1, "name")
#sheet.append_row(["name", "pubkey","statement","signature"])
#sheet.__dir__()

On this layer we meet to post:

- `name` of participant
- `pubkey` public key of participant in hex
- `statement` is the declaration signed by participant 
- `signature` is the ecc signature of the statement

In [ ]:
my_name=input('input name')
my_statement=input('input statement')
open('llaves/statement1.txt','wb').write(my_statement.encode())

In addition we use the python scripts to perform the cryptography

In [ ]:
! cat llaves/statement1.txt

In [ ]:
ret = !{ 'python Colegio_Invisible/scripts/ecc_sign.py llaves/prvkey1.eck llaves/statement1.txt llaves/sig1.sig'}
sig1=ret[-1]
sig1

Here we post to the Worksheet to create a roster of all the participants

In [ ]:
sheet.append_row([my_name,pubkey1,my_statement,sig1])

This is a bird sound

In [ ]:
import IPython
IPython.display.Audio("Colegio_Invisible/sound/bird2.wav")

Reading the data a Pandas Dataframe

In [ ]:
import pandas as pd

dataframe = pd.DataFrame(sheet.get_all_records())
dataframe

We can verify statements. Here we use the `ecies` cryptography library directly without using the external scripts. 

In [ ]:
import eth_keys
import ecies

eth_keys.datatypes.Signature(bytes.fromhex(sig1)).verify_msg(my_statement.encode(),eth_keys.keys.PublicKey(bytes.fromhex(pubkey1)))

In [ ]:
def verify(sig,statement,pubkey):
  return eth_keys.datatypes.Signature(bytes.fromhex(sig)).verify_msg(statement.encode() if type(statement)!=bytes else statement,eth_keys.keys.PublicKey(bytes.fromhex(pubkey)))

In [ ]:
verify(sig1,my_statement,pubkey1)

Here we verify all rows of the roster

In [ ]:
dataframe.set_index('name').apply(lambda row: verify(row.signature,row.statement,row.pubkey),axis=1)

Collecting the names of all the participants

In [ ]:
participants=dataframe.name.tolist()
participants

Doing variable assignment for needed Worksheets

In [ ]:
sheet1=sheets.worksheets()[1]
sheet2=sheets.worksheets()[2]
sheet3=sheets.worksheets()[3]
sheet1,sheet2,sheet3

In [ ]:
#ONLY FOR DDOG MC
#df=pd.DataFrame('dada',columns=participants,index=participants)
#sheet1.clear()
#sheet1.update([df.columns.values.tolist()] + df.values.tolist())
#sheet2.clear()
#sheet2.update([df.columns.values.tolist()] + df.values.tolist())
#sheet3.clear()
#sheet3.update([df.columns.values.tolist()] + df.values.tolist())
#df

## Worksheet 1

This is a place for casual public chats
- row is sender
- column is recipient

In [ ]:
dataframe1 = pd.DataFrame(sheet1.get_all_records())
dataframe1.index=dataframe1.columns
dataframe1

Make a little recipient widget interface

In [ ]:
import ipywidgets as widgets
w=widgets.Dropdown(
    options=participants,
    value='DrDoeg',
    description='Recipient:',
    disabled=False,
)

In [ ]:
#have a look
w

In [ ]:
w.value

In [ ]:
my_name

Creating messaging

In [ ]:
p_dict=dict([ (e,i) for i,e in enumerate(participants)])
p_dict

Select your message recipient using the widget.

In [ ]:
print(f'You are {my_name}')
print('Select a message recipient')
w

Write message 

Post to spreadsheet

refresh dataframe from worksheet

In [ ]:
message=input('send a message')
write_cell=(p_dict[my_name]+2,p_dict[w.value]+1)
sheet1.update_cell(*write_cell, message)
dataframe1 = pd.DataFrame(sheet1.get_all_records())
dataframe1.index=dataframe1.columns
dataframe1

In [ ]:
dataframe1 = pd.DataFrame(sheet1.get_all_records())
dataframe1.index=dataframe1.columns
dataframe1

In [ ]:
dataframe1[my_name]

# Encrypted Messaging

In [ ]:
dataframe2 = pd.DataFrame(sheet2.get_all_records())
dataframe2.index=dataframe2.columns
dataframe2

In [ ]:
def encrypt_message(hexPubKey,message):
    if type(message)!=bytes:
        message=message.encode()
    return ecies.encrypt(hexPubKey,message).hex()

In [ ]:
w

In [ ]:
encrypt_message(dataframe.set_index('name').pubkey[w.value],'hey guys hey hey')

In [ ]:
message=input('send an encrypted message')
write_cell=(p_dict[my_name]+2,p_dict[w.value]+1)
sheet2.update_cell(*write_cell, encrypt_message(dataframe.set_index('name').pubkey[w.value],message))
dataframe2 = pd.DataFrame(sheet2.get_all_records())
dataframe2.index=dataframe2.columns
dataframe2

In [ ]:
dataframe2 = pd.DataFrame(sheet2.get_all_records())
dataframe2.index=dataframe2.columns
dataframe2

In [ ]:
def import_privKey(path,password):
  import hashlib
  if type(password)!=bytes:
    password=password.encode()
  f=open(path,'rb')
  decrypted_bytes=ecies.aes_decrypt(key=hashlib.sha256(password).digest(),cipher_text=f.read())
  privKey=eth_keys.keys.PrivateKey(decrypted_bytes)
  f.close()
  return privKey

def decrypt_message(privKey,cipherhex):
    ciphertext=bytes.fromhex(cipherhex)
    try:
      return ecies.decrypt(privKey.to_hex(),ciphertext)
    except:
      return None

In [ ]:
decrypt_message(import_privKey('llaves/prvkey1.eck',''),dataframe2[my_name][my_name])


In [ ]:
dataframe2 = pd.DataFrame(sheet2.get_all_records())
dataframe2.index=dataframe2.columns
dataframe2

k=import_privKey('llaves/prvkey1.eck','')
tr=dataframe2[my_name].apply(lambda x: decrypt_message(k,x))
tr

In [ ]:
def sign_message(privKey,message):
    if type(message)!=bytes:
        message=message.encode()
    signature= privKey.sign_msg(message)
    return signature.to_bytes().hex()

In [ ]:
mess=input('message')
mess_sig=sign_message(import_privKey('llaves/prvkey1.eck',''),mess)
mess_sig

In [ ]:
emess=encrypt_message(pubkey1,mess)
emess

In [ ]:
emess_sig=sign_message(k,bytes.fromhex(emess))
emess_sig,len(emess_sig)

In [ ]:
emess_sig,emess

In [ ]:
def sign_encrypt_message(privKey,hexpubKey,message):
    if type(message)!=bytes:
        message=message.encode()
    byte_enc=bytes.fromhex(encrypt_message(hexpubKey,message))
    signature= privKey.sign_msg(byte_enc)
    return signature.to_bytes().hex()+byte_enc.hex()

In [ ]:
sgen=sign_encrypt_message(k,pubkey1,mess)
sgen

In [ ]:
def decrypt_verify(pubkey,privkey,block):
  try:
    ver=verify(block[:130],bytes.fromhex(block[130:]),pubkey)
    return decrypt_message(privkey,block[130:]),ver
  except:
    return None

In [ ]:
decrypt_verify(pubkey1,k,sgen)

In [ ]:
message=input('send a signed and encrypted message')
write_cell=(p_dict[my_name]+2,p_dict[w.value]+1)
signed_enc_message=sign_encrypt_message(k,pubkey1,message)
sheet3.update_cell(*write_cell, encrypt_message(dataframe.set_index('name').pubkey[w.value],signed_enc_message))
dataframe3 = pd.DataFrame(sheet3.get_all_records())
dataframe3.index=dataframe3.columns
dataframe3

In [ ]:
dataframe3 = pd.DataFrame(sheet3.get_all_records())
dataframe3.index=dataframe3.columns
dataframe3

In [ ]:
pubkey_name=dataframe.set_index('name')['pubkey']
dataframe3[[my_name]].reset_index().apply(lambda x: decrypt_verify(pubkey_name[x['index']],k,x[my_name])  ,axis=1)